In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.spatial
import os as os

import warnings
warnings.filterwarnings('ignore')

In [2]:
# inputs: empty array of 593 frames x 68 AAMs 
# outputs: 593 frames x 64 AUs array of 0s
aam = [[0]*136 for _ in range(593)] 
aam0 = [[0]*136 for _ in range(593)] 
aamvec = [[0]*136 for _ in range(593)] 
au = [[0]*64 for _ in range(593)] 
emotion = [[0]*7 for _ in range(593)]
# aam = []
# au = []
# emotion = []

# au2d looks like:
# AUs     0   1   2  ...  63
# ------------------------
# frame 
# 0     | 0 | 1 | 0 | ... 
# ------------------------
# 1     | 0 | 1 | 1 | ... 
# ------------------------
# 2     | 1 | 1 | 0 | ... 
# ------------------------
# ...
# 592

# Open and read the output dataset (AU)
for i, filename in enumerate(os.listdir("au")):
    # first i is .DS_store
    if filename.endswith(".txt"):
        with open("au/" + filename) as f:
            lines = f.readlines()
            for line in lines:
                els = line.split()
                if els:
                    if els[0][len(els[0]) - 1] == "0":
                        au[i - 1][int(els[0][0]) - 1] = 1
                    else:
                        au[i - 1][int(els[0][0] + els[0][2]) - 1] = 1
            f.close()

# for frame in au2d:
#     au.append([feature for feature in frame])

In [3]:
# inputs looks like: 
# AAMs         0           1      ... 67
# -------------------------------------
# frame 
# 0     | 1.2 | 3.4 | .2 | 3 | ...
# -------------------------------------
# 1     | [6.5, 3] | [2.2, 3.1] | ...
# -------------------------------------
# 2     | [5.4, 2.2] | [.9, 2.9] | ...
# -------------------------------------
# ...
# 592

for i, filename in enumerate(os.listdir("aam")):
    # first i is .DS_store
    if filename.endswith(".txt"):
        with open("aam/" + filename) as f:
            lines = f.readlines()
            for j, line in enumerate(lines):
                els = line.split()
                if els:
                    aam[i - 1][2 * j] = float(els[0])
                    aam[i - 1][2 * j + 1] = float(els[1])
#                     aam[i - 1][j] = [float(els[0]), float(els[1])]
            f.close()
for i, filename in enumerate(os.listdir("aamvec")):
    # first i is .DS_store
    if filename.endswith(".txt"):
        with open("aamvec/" + filename) as f:
            lines = f.readlines()
            for j, line in enumerate(lines):
                els = line.split()
                if els:
                    aam0[i - 1][2 * j] = float(els[0])
                    aam0[i - 1][2 * j + 1] = float(els[1])
#                     aam[i - 1][j] = [float(els[0]), float(els[1])]
            f.close()

In [4]:
# only 327 out of 593 are coded
# non-coded ones are given -1 as class
# 0 = neutral
# 1 = anger
# 2 = contempt
# 3 = disgust
# 4 = fear
# 5 = happy
# 6 = sadness
# 7 = surprise

for i, filename in enumerate(os.listdir("emo")):
    # first i is .DS_store
    if filename.endswith(".txt"):
        with open("emo/" + filename) as f:
            lines = f.readlines()
            for line in lines:
                els = line.split() # except last character
                if els:
                    if (int(float(els[0])) == -1):
                        for e in range(7):
                            emotion[i-1][e] = -1
                    else: 
                        emotion[i - 1][int(float(els[0])) - 1] = 1
            f.close()
            
# for frame in emotion2d:
#     emotion.append(frame)

In [5]:
for i in range(593):
    for j in range(136):
        aamvec[i][j] = float("%.1f" % (aam[i][j] - aam0[i][j]))


In [6]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcess
from sklearn.cross_decomposition import CCA
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.semi_supervised import LabelPropagation

cl_lr = LinearRegression()
cl_lda = LinearDiscriminantAnalysis()
cl_krg = KernelRidge()
cl_svm = SVC()
cl_sgd = SGDClassifier()
cl_knn5 = KNeighborsClassifier(n_neighbors=5)
cl_knn10 = KNeighborsClassifier(n_neighbors=10)
cl_knn15 = KNeighborsClassifier(n_neighbors=15)
cl_gp = GaussianProcess()
cl_cd = CCA()
cl_nb = GaussianNB()
cl_dt = DecisionTreeClassifier()
cl_ab = AdaBoostClassifier()
cl_mc = OneVsRestClassifier(SVC(kernel='linear'))
cl_lp = LabelPropagation()
cl_ir = IsotonicRegression()

In [7]:
kf = KFold(n=593, n_folds=593)
train_input = []
train_output = []

vlr = 0
vlda = 0
vkrg = 0
vsvm = 0
vsgd = 0
vknn5 = 0
vknn10 = 0
vknn15 = 0
vgp = 0
vcd = 0
vnb = 0
vdt = 0
vab = 0
vmc = 0
vlp = 0
vir = 0

In [8]:
# train_input = []
# train_output = []
# for (train_index, test_index) in kf:
#     for idx in train_index:
#         if (emotion[idx] != -1):
#             train_input.append(au[idx])
#             train_output.append(emotion[idx])
#     cl_lr.fit(train_input, train_output)
#     vlr += cl_lr.score(au[test_index], [emotion[test_index]])
    
#     train_input = []
#     train_output = []

In [8]:
warnings.filterwarnings('ignore')
train_input = []
train_output = []
vlda = 0
vsvm = 0
vsgd = 0
vknn5 = 0
vknn10 = 0
vknn15 = 0
vnb = 0
vdt = 0
vab = 0
vlp = 0
for (train_index, test_index) in kf:
    for idx in train_index:
        if (emotion[idx][0] != -1):
            train_input.append(au[idx])
    for idxemo in range(7):
        for idx in train_index:
            if (emotion[idx][0] != -1):                
                train_output.append(emotion[idx][idxemo])
        cl_lda.fit(train_input, train_output)
        vlda += cl_lda.score(au[test_index], [emotion[test_index][idxemo]])    
        cl_svm.fit(train_input, train_output)
        vsvm += cl_svm.score(au[test_index], [emotion[test_index][idxemo]]) 
        cl_sgd.fit(train_input, train_output)
        vsgd += cl_sgd.score(au[test_index], [emotion[test_index][idxemo]])
        cl_knn5.fit(train_input, train_output)
        vknn5 += cl_knn5.score(au[test_index], [emotion[test_index][idxemo]])   
        cl_knn10.fit(train_input, train_output)
        vknn10 += cl_knn10.score(au[test_index], [emotion[test_index][idxemo]])   
        cl_knn15.fit(train_input, train_output)
        vknn15 += cl_knn15.score(au[test_index], [emotion[test_index][idxemo]])   
        cl_nb.fit(train_input, train_output)
        vnb += cl_nb.score(au[test_index], [emotion[test_index][idxemo]])
        cl_dt.fit(train_input, train_output)
        vdt += cl_dt.score(au[test_index], [emotion[test_index][idxemo]])  
        cl_ab.fit(train_input, train_output)
        vab += cl_ab.score(au[test_index], [emotion[test_index][idxemo]])
        cl_lp.fit(train_input, train_output)
        vlp += cl_lp.score(au[test_index], [emotion[test_index][idxemo]])
        train_output = []
    train_input = []


In [10]:
# train_input = []
# train_output = []
# for (train_index, test_index) in kf:
#     for idx in train_index:
#         if (emotion[idx] != -1):
#             train_input.append(au[idx])
#             train_output.append(emotion[idx])
        
#     cl_krg.fit(train_input, train_output) 
#     vkrg += cl_krg.score(au[test_index], [emotion[test_index]])
    
#     train_input = []
#     train_output = []

In [11]:
train_input = []
train_output = []
for (train_index, test_index) in kf:
    for idx in train_index:
        if (emotion[idx] != -1):
            train_input.append(au[idx])
            train_output.append(emotion[idx])
        
    cl_svm.fit(train_input, train_output)
    vsvm += cl_svm.score(au[test_index], [emotion[test_index]])   
    
    train_input = []
    train_output = []

In [12]:
train_input = []
train_output = []
for (train_index, test_index) in kf:
    for idx in train_index:
        if (emotion[idx] != -1):
            train_input.append(au[idx])
            train_output.append(emotion[idx])
        
    cl_sgd.fit(train_input, train_output)
    vsgd += cl_sgd.score(au[test_index], [emotion[test_index]])
    
    train_input = []
    train_output = []

In [13]:
train_input = []
train_output = []
for (train_index, test_index) in kf:
    for idx in train_index:
        if (emotion[idx] != -1):
            train_input.append(au[idx])
            train_output.append(emotion[idx])
        
    cl_knn5.fit(train_input, train_output)
    vknn5 += cl_knn5.score(au[test_index], [emotion[test_index]])   
    
    train_input = []
    train_output = []

In [14]:
train_input = []
train_output = []
for (train_index, test_index) in kf:
    for idx in train_index:
        if (emotion[idx] != -1):
            train_input.append(au[idx])
            train_output.append(emotion[idx])
        
    cl_knn10.fit(train_input, train_output)
    vknn10 += cl_knn10.score(au[test_index], [emotion[test_index]])   
    
    train_input = []
    train_output = []

In [15]:
train_input = []
train_output = []
for (train_index, test_index) in kf:
    for idx in train_index:
        if (emotion[idx] != -1):
            train_input.append(au[idx])
            train_output.append(emotion[idx])
        
    cl_knn15.fit(train_input, train_output) 
    vknn15 += cl_knn15.score(au[test_index], [emotion[test_index]])   
    
    train_input = []
    train_output = []

In [16]:
# train_input = []
# train_output = []
# for (train_index, test_index) in kf:
#     for idx in train_index:
#         if (emotion[idx] != -1):
#             train_input.append(au[idx])
#             train_output.append(emotion[idx])
        
#     cl_cd.fit(train_input, train_output)
#     vcd += cl_cd.score(au[test_index], [emotion[test_index]])   
    
#     train_input = []
#     train_output = []

In [17]:
train_input = []
train_output = []
for (train_index, test_index) in kf:
    for idx in train_index:
        if (emotion[idx] != -1):
            train_input.append(au[idx])
            train_output.append(emotion[idx])
        
    cl_nb.fit(train_input, train_output)
    vnb += cl_nb.score(au[test_index], [emotion[test_index]])
    
    train_input = []
    train_output = []

In [18]:
train_input = []
train_output = []
for (train_index, test_index) in kf:
    for idx in train_index:
        if (emotion[idx] != -1):
            train_input.append(au[idx])
            train_output.append(emotion[idx])
        
    cl_dt.fit(train_input, train_output)
    vdt += cl_dt.score(au[test_index], [emotion[test_index]])   

    train_input = []
    train_output = []

In [19]:
train_input = []
train_output = []
for (train_index, test_index) in kf:
    for idx in train_index:
        if (emotion[idx] != -1):
            train_input.append(au[idx])
            train_output.append(emotion[idx])
        
    cl_ab.fit(train_input, train_output)
    vab += cl_ab.score(au[test_index], [emotion[test_index]])
    
    train_input = []
    train_output = []

In [20]:
train_input = []
train_output = []
for (train_index, test_index) in kf:
    for idx in train_index:
        if (emotion[idx] != -1):
            train_input.append(au[idx])
            train_output.append(emotion[idx])
        
    cl_mc.fit(train_input, train_output)
    vmc += cl_mc.score(au[test_index], [emotion[test_index]]) 
    
    train_input = []
    train_output = []

ValueError: Found arrays with inconsistent numbers of samples: [ 1 64]

In [21]:
train_input = []
train_output = []
for (train_index, test_index) in kf:
    for idx in train_index:
        if (emotion[idx] != -1):
            train_input.append(au[idx])
            train_output.append(emotion[idx])
        
    cl_lp.fit(train_input, train_output)
    vlp += cl_lp.score(au[test_index], [emotion[test_index]])
    
    train_input = []
    train_output = []

In [22]:
train_input = []
train_output = []
for (train_index, test_index) in kf:
    for idx in train_index:
        if (emotion[idx] != -1):
            train_input.append(au[idx])
            train_output.append(emotion[idx])
        
    cl_ir.fit(train_input, train_output)
    vir += cl_ir.score(au[test_index], [emotion[test_index]]) 
    
    train_input = []
    train_output = []

ValueError: X should be a 1d array

In [9]:
# print("Linear regression: " + str(vlr / (len(train_input) - 1.)))
print("LDA: " + str(vlda / 2288.))
# print("KRG: " + str(vkrg / (len(train_input) - 1.)))
print("SVM: " + str(vsvm / 2288.))
print("SGD: " + str(vsgd / 2288.))
print("KNN5: " + str(vknn5 / 2288.))
print("KNN10: " + str(vknn10 / 2288.))
print("KNN15: " + str(vknn15 / 2288.))
# print("GP: " + str(vgp / (len(train_input) - 1.)))
# print("CD: " + str(vcd / (len(train_input) - 1.)))
print("NB: " + str(vnb / 2288.))
print("DT: " + str(vdt / 2288.))
print("adaboost: " + str(vab / 2288.))
# print("MC: " + str(vmc / (len(train_input) - 1.)))
print("LP: " + str(vlp / 2288.))
# print("IR: " + str(vir / (len(train_input) - 1.)))

LDA: 0.968531468531
SVM: 0.946241258741
SGD: 0.956293706294
KNN5: 0.972027972028
KNN10: 0.97027972028
KNN15: 0.964597902098
NB: 0.693181818182
DT: 0.958916083916
adaboost: 0.963286713287
LP: 0.968968531469


In [43]:
print (len(train_input) - 1.)

325.0


In [ ]:
# for (train_index, test_index) in kf:
#     for idx in train_index:
#         train_input.append(aam[idx])
#         train_output.append(emotion[idx])
        
#     cl_lr.fit(train_input, train_output)
#     cl_lda.fit(train_input, train_output)
#     cl_krg.fit(train_input, train_output)
#     cl_svm.fit(train_input, train_output)
#     cl_sgd.fit(train_input, train_output)
#     cl_knn5.fit(train_input, train_output)
#     cl_knn10.fit(train_input, train_output)
#     cl_knn15.fit(train_input, train_output)
# #     cl_gp.fit(train_input, train_output)
#     cl_cd.fit(train_input, train_output)
#     cl_nb.fit(train_input, train_output)
#     cl_dt.fit(train_input, train_output)
#     cl_ab.fit(train_input, train_output)
#     cl_mc.fit(train_input, train_output)
#     cl_lp.fit(train_input, train_output)
#     cl_ir.fit(train_input, train_output)
    
#     vlr += cl_lr.score(au[test_index], [emotion[test_index]])
#     vlda += cl_lda.score(aam[test_index], [emotion[test_index]])    
#     vkrg += cl_krg.score(aam[test_index], [emotion[test_index]])
#     vsvm += cl_svm.score(aam[test_index], [emotion[test_index]])   
#     vsdg += cl_sdg.score(aam[test_index], [emotion[test_index]])
#     vknn5 += cl_knn.score(aam[test_index], [emotion[test_index]])   
#     vknn10 += cl_knn.score(aam[test_index], [emotion[test_index]])   
#     vknn15 += cl_knn.score(aam[test_index], [emotion[test_index]])   
# #     vgp += cl_gp.score(aam[test_index], [emotion[test_index]])
#     vcd += cl_cd.score(aam[test_index], [emotion[test_index]])   
#     vnb += cl_nb.score(aam[test_index], [emotion[test_index]])
#     vdt += cl_dt.score(aam[test_index], [emotion[test_index]])   
#     vab += cl_ab.score(aam[test_index], [emotion[test_index]])
#     vmc += cl_mc.score(aam[test_index], [emotion[test_index]]) 
#     vlp += cl_lp.score(aam[test_index], [emotion[test_index]])
#     vir += cl_ir.score(aam[test_index], [emotion[test_index]]) 
    
#     train_input = []
#     train_output = []